## Dados de metricas

|           Métrica                            |Valor |
|:--------------------------------------------:|:----:|
|          Taxa de juros do rotativo           |  17% |
|          Taxa de intercâmbio                 |  5%  |
|            Custo unitário do cartão          |  10  |
|Custo por minuto do serviço de atendimento    |  2.5 |
|           Inflação mensal                    | 0.5% |


## Dados de comportamento


|           Coluna                             |Tipo     | Descrição |
|:--------------------------------------------:|:----:| |
|ids|String|identificador único de um aplicante|
|credit_line|  String | Limite do cartão|
|month|Int| Ordenação dos meses que a pessoa é cliente, sendo 0 o primeiro mês dela como cliente|
|spend|Float|Valor gasto naquele mês|
|revolving_balance|Float|Valor que o cliente não pagou da fatura atual e que irá rolar para a próxima |
|card_request| Int |Se o cliente solicitou uma nova via do cartão (ou a primeira) |
| minutes_cs|Float|Quantidade de minutos utilizados do serviço de atendimento ao consumidor |

## Premissas

Algumas premissas que serão levadas em conta como forma de garantir a fundamentação dos cálculos e modelos.

### Dados
- O mês 0 dos dados de comportamento é o mesmo mês real para todos clientes
- Um cliente marcado como fraude e inadimplência terá o prejuizo calculado como se fosse um caso de fraude

### Balanço do rotativo
- O pagamento do rotativo será considerado como se tivesse ocorrido 

### Fraude
- Se um cliente é fraudador, ele dará um prejuízo igual ao limite de crédito concedido.
- Todo os histórico dele sera desconsiderado

### Inadimplência
- No caso de inadimplência o prejuízo é a soma do montante da última fatura mais o balanço do rotativo do último mês



In [11]:
import pandas as pd
import numpy as np
import sys
from src.data import make_data

In [12]:
sys.path.append("..")

In [13]:
spend_agg = make_data.spend_agg(False)

2018-10-06 21:59:05,931 INFO     Reading spend
2018-10-06 21:59:06,652 INFO     spend_train shape: (980818, 7)
2018-10-06 21:59:06,652 INFO     Reading acquisition
2018-10-06 21:59:07,186 INFO     acquisition_train shape: (45000, 43)
2018-10-06 21:59:07,186 INFO     Creating new columns.
2018-10-06 21:59:07,875 INFO     Returning spend_base
2018-10-06 21:59:08,090 INFO     Returning spend_agg


In [17]:
# def business_model():
decision = pd.DataFrame(index=spend_agg.index, columns=['APPROVE', 'LIMIT'])

,APPROVE,LIMIT
ids,,
0003da40-123b-031a-63b3-f7c9f18516ab,NaN,NaN
0004a450-07bd-c652-8473-4e9dedcff8b8,NaN,NaN
0005967a-2048-6197-a054-30954efe93b1,NaN,NaN
0009b1d5-f5c3-428e-63dd-39b5f04ad7df,NaN,NaN
0010b103-a9e4-e08f-ce04-0c65e71a6df9,NaN,NaN
0010e9b7-c5b2-7944-4ddb-88dc5ad6da65,NaN,NaN
00141481-d30b-2cb1-fefc-dae26098bda6,NaN,NaN
0015133f-867d-7086-a0ff-d50e95c4a615,NaN,NaN
00179c95-2064-da76-3de6-b3d3c40e05cb,NaN,NaN
